In [1]:
!pip install mtcnn -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.8 MB/s eta 0:00:00


In [2]:
!pip install onnxruntime -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.2 MB/s eta 0:00:00


In [3]:
pip install insightface opencv-python scikit-learn matplotlib joblib -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 6.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 95.9 MB/s eta 0:00:00


In [4]:
import cv2
import numpy as np
import joblib
import matplotlib.pyplot as plt
from insightface.app import FaceAnalysis
from sklearn.metrics.pairwise import cosine_similarity


/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.7' (you have '2.0.6'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [5]:
def load_model(ctx_id=0):
    face_app = FaceAnalysis(name='buffalo_l')  # ArcFace + MTCNN
    face_app.prepare(ctx_id=ctx_id)  # 0 for CPU, 1 for GPU
    return face_app


In [6]:
def load_embeddings(embedding_path, label_path):
    embeddings = joblib.load(embedding_path)
    labels = joblib.load(label_path)
    return embeddings, labels


In [10]:
def recognize_faces(image_path, embeddings, labels, face_app, threshold=0.5):
    img = cv2.imread(image_path)
    faces = face_app.get(img)

    if not faces:
        print("No faces detected.")
        return [], []

    detected_labels = []
    unknown_faces = 0
    face_results = []

    for face in faces:
        emb = face.embedding
        bbox = np.array(face.bbox, dtype=int)

        sims = cosine_similarity([emb], embeddings)
        best_match_idx = np.argmax(sims)
        best_score = sims[0][best_match_idx]
        predicted_label = labels[best_match_idx]

        if best_score >= threshold:
            label_text = f"{predicted_label} ({best_score:.2f})"
            detected_labels.append(predicted_label)
        else:
            label_text = f"Unknown ({best_score:.2f})"
            predicted_label = "Unknown"
            unknown_faces += 1


        # Annotate bounding box and label
        x1, y1, x2, y2 = bbox
        color = (0, 255, 255) if predicted_label != "Unknown" else (0, 0, 255)  # Yellow for known, Red for unknown
        cv2.rectangle(img, (x1, y1), (x2, y2), color, thickness=3)

        # Draw label background
        label_size, _ = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)
        label_width, label_height = label_size
        cv2.rectangle(img, (x1, y1 - label_height - 10), (x1 + label_width + 5, y1), color, cv2.FILLED)

        # Put label text
        cv2.putText(img, label_text, (x1 + 2, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)
        # Print label with score
        print(f"Detected: {label_text}")
        face_results.append((predicted_label, best_score, bbox.tolist()))

    # Show final image
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(12, 10))
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.title("Recognized Faces")
    plt.show()

    # Determine present vs absent
    all_known_labels = set(labels)
    detected_labels_set = set(detected_labels)
    absent_labels = sorted(all_known_labels - detected_labels_set)
    present_labels = sorted(detected_labels_set)

    print("\n--- Attendance Report ---")
    print(f"✅ Present ({len(present_labels)}): {present_labels}")
    print(f"❌ Absent ({len(absent_labels)}): {absent_labels}")
    if unknown_faces > 0:
        print(f"❓ Unknown Faces: {unknown_faces}")

    return face_results, present_labels


#group img 1 : ground truth : 5451 5453: path : /content/drive/MyDrive/Mini/group images/group img 1.jpg
#group img 4 : ground truth: 5440 5461 path: /content/drive/MyDrive/Mini/group images/group img 4.jpg
#group_test_1 : ground truth: 5413 5452 5451 5409 5423 5449 5445 path : /content/drive/MyDrive/Mini/group images/group_test_1.jpg
#group_test_2: ground truth: 5413 5452 5451 5409 5435 5423 5449 5445 path: /content/drive/MyDrive/Mini/group images/group_test_2.jpg


In [7]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

# Ground truth for test images
TEST_IMAGES = [
    ("/content/drive/MyDrive/Mini/group images/group img 1.jpg", ["5451", "5453"]),
    ("/content/drive/MyDrive/Mini/group images/group img 4.jpg", ["5440", "5461"]),
    ("/content/drive/MyDrive/Mini/group images/group_test_1.jpg", ["5413", "5452", "5451", "5409", "5423", "5449", "5445"]),
    ("/content/drive/MyDrive/Mini/group images/group_test_2.jpg", ["5413", "5452", "5451", "5409", "5435", "5423", "5449", "5445"]),
]

def evaluate_model_on_tests(embeddings, labels, face_app, threshold=0.7):
    results = []

    for img_path, ground_truth in TEST_IMAGES:
        print(f"\n🧪 Testing: {img_path.split('/')[-1]}")
        face_results, detected = recognize_faces(
            image_path=img_path,
            embeddings=embeddings,
            labels=labels,
            face_app=face_app,
            threshold=threshold
        )

        # Metrics
        correct = set(detected) & set(ground_truth)
        acc = len(correct) / len(ground_truth)

        y_true = [1 if label in ground_truth else 0 for label in labels]
        y_pred = [1 if label in detected else 0 for label in labels]

        precision = precision_score(y_true, y_pred, zero_division=0)
        recall = recall_score(y_true, y_pred, zero_division=0)
        f1 = f1_score(y_true, y_pred, zero_division=0)

        print(f"✅ Accuracy: {acc:.2f}, 🎯 Precision: {precision:.2f}, 🔁 Recall: {recall:.2f}, 🧮 F1: {f1:.2f}")

        results.append({
            "image": img_path.split('/')[-1],
            "accuracy": acc,
            "precision": precision,
            "recall": recall,
            "f1": f1
        })

    return results


def plot_metrics(results):
    metrics = ["accuracy", "precision", "recall", "f1"]
    x_labels = [r["image"] for r in results]
    x = np.arange(len(x_labels))
    width = 0.2

    # Set up the figure with high DPI (300) for publication quality
    plt.figure(figsize=(12, 6), dpi=300)

    # Update font sizes for clarity
    plt.rc('font', size=12)
    plt.rc('axes', titlesize=16, labelsize=14)
    plt.rc('xtick', labelsize=12)
    plt.rc('ytick', labelsize=12)
    plt.rc('legend', fontsize=12)

    # Plot each metric
    for i, metric in enumerate(metrics):
        plt.bar(x + i * width, [r[metric] for r in results], width, label=metric.capitalize())

    plt.xticks(x + 1.5 * width, x_labels, rotation=15)
    plt.xlabel("Group Image", fontsize=14)
    plt.ylabel("Score", fontsize=14)
    plt.title("Face Recognition Performance per Group Image", fontsize=16)
    plt.ylim(0, 1.05)
    plt.legend(loc="best")
    plt.grid(axis="y", linestyle="--", alpha=0.5)
    plt.tight_layout()

    # Save the high DPI figure for publication quality output
    save_path = '/content/drive/MyDrive/Mini/performance_metrics_paper_ready.png'
    plt.savefig(save_path, bbox_inches='tight', dpi=300)
    print(f"📊 High DPI performance metrics graph saved to {save_path}")

    plt.show()


In [ ]:
def main():
    #File paths
    EMB_PATH = '/content/drive/MyDrive/Mini/embeddings.pkl'
    LABEL_PATH = '/content/drive/MyDrive/Mini/labels.pkl'
    IMG_PATH = '/content/drive/MyDrive/Mini/group images/group_test_2.jpg'

    #Load models and data
    face_app = load_model(ctx_id=0)
    embeddings, labels = load_embeddings(EMB_PATH, LABEL_PATH)

    # Run recognition
    recognize_faces(IMG_PATH, embeddings, labels, face_app, threshold=0.7)

In [8]:
def main():
    EMB_PATH = '/content/drive/MyDrive/Mini/embeddings.pkl'
    LABEL_PATH = '/content/drive/MyDrive/Mini/labels.pkl'

    print("📦 Loading ArcFace model...")
    face_app = load_model(ctx_id=0)

    print("📂 Loading embeddings and labels...")
    embeddings, labels = load_embeddings(EMB_PATH, LABEL_PATH)

    print("🚀 Starting Evaluation...")
    results = evaluate_model_on_tests(embeddings, labels, face_app, threshold=0.7)

    print("📊 Generating Graphs...")
    plot_metrics(results)

In [11]:
if __name__ == "__main__":
  main()

Output hidden; open in https://colab.research.google.com to view.

In [12]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import roc_curve, auc
import shutil
import os

In [13]:
import cv2
import pickle
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, auc
from sklearn.metrics.pairwise import cosine_similarity
import shutil
import os

# Existing functions (unchanged):
# - load_model
# - load_embeddings
# - plot_metrics (not called)
# - collect_roc_data
# - TEST_IMAGES
# - evaluate_model_on_tests
# - recognize_faces

# Updated plot_roc_curve with enhancements for publication
def plot_roc_curve(true_labels, sim_scores):
    try:
        fpr, tpr, thresholds = roc_curve(true_labels, sim_scores, pos_label=1)
        roc_auc = auc(fpr, tpr)
        eer = fpr[np.argmin(np.abs(fpr - (1 - tpr)))] if len(fpr) > 0 else 0
        eer_tpr = 1 - eer  # TPR at EER

        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, color='blue', lw=3, label=f'ROC Curve (AUC = {roc_auc:.3f})')  # Thicker line, AUC to 3 decimals
        plt.plot([0, 1], [0, 1], color='gray', linestyle='--', lw=1)
        plt.scatter([eer], [eer_tpr], color='red', marker='o', s=100, label=f'EER = {eer:.3f}')
        # Add label next to EER marker
        plt.text(eer + 0.02, eer_tpr - 0.05, f'EER = {eer:.3f}', fontsize=10, color='red')

        plt.xlim([0.0, 0.2])  # Zoom in on low FPRs
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate (FPR)', fontsize=12)
        plt.ylabel('True Positive Rate (TPR)', fontsize=12)
        plt.title('ROC Curve for Face Recognition on Test Group Images', fontsize=14, pad=15)
        plt.legend(loc='lower right', fontsize=11)
        plt.grid(True, linestyle='--', alpha=0.5)  # Lighter grid
        plt.tight_layout()

        # Save the plot
        save_path = '/content/drive/MyDrive/Mini/roc_curve7.png'
        plt.savefig(save_path, bbox_inches='tight', dpi=300)  # Higher DPI for publication
        print(f"📈 ROC curve saved to {save_path}")

        # Display the plot in Colab
        plt.show()
        plt.close()

        # Verify file exists
        if os.path.exists(save_path):
            print(f"✅ Verified: ROC curve file exists at {save_path}")
        else:
            print(f"❌ Error: ROC curve file not found at {save_path}")

        return roc_auc, eer
    except Exception as e:
        print(f"Error generating ROC curve plot: {e}")
        return None, None
